In [3]:
import os
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=os.environ.get("OPENAI_API_KEY"))


def setup_es_client() -> Elasticsearch:
    """
    Set up and return an Elasticsearch client using environment variables.

    This function reads the Elasticsearch API key and endpoint from environment variables,
    initializes an Elasticsearch client, and returns it.

    Returns:
        Elasticsearch: An initialized Elasticsearch client.
    """
    # Load environment variables for Elasticsearch API and endpoint
    ES_API = os.environ.get("ES_API")
    ES_ENDPOINT = os.environ.get("ES_ENDPOINT")

    # Initialize the Elasticsearch client
    es_client = Elasticsearch(
        ES_ENDPOINT,
        api_key=ES_API
    )

    return es_client


# Create the Elasticsearch client by calling the setup function
es_client = setup_es_client()


def setup_es_store(index_name: str, es_client: Elasticsearch):
    """
       Sets up and returns an ElasticsearchStore index for storing embeddings and for RAG.

       Args:
           index_name (str): The name of the Elasticsearch index where embeddings will be stored.
           es_client (Elasticsearch): An instance of the Elasticsearch client connected to the Elasticsearch cluster.

       Returns:
           ElasticsearchStore: An instance of ElasticsearchStore configured with the provided index name,
                                embeddings, and Elasticsearch connection.

       """
    es_vector_store = ElasticsearchStore(
        embedding=embeddings,
        index_name=index_name,
        es_connection=es_client
    )
    return es_vector_store

In [4]:
# Setup Elasticsearch store
es_store = setup_es_store("azal_activities", es_client)


In [29]:
def get_azal_activities(question:str):
    """
    Fetch general activities based on a query from Elasticsearch store and process the response.
    """
    # Fetch documents matching the query and location from Elasticsearch
    # docs = es_store.as_retriever(
    #         question
    #         # k=4
    #     )

        # return response
    # return docs
    docs = es_store.as_retriever().invoke(question)
    return docs

In [31]:
get_azal_activities("what is azal")

[Document(page_content="Embark on a culinary adventure through Izmir's trendy Alsancak district. Sample local street food, visit traditional eateries, and learn about Aegean cuisine.", metadata={'location': 'izmir', 'activity_address': 'alsancak, 35220 konak/i̇zmir', 'activity_title': 'alsancak food tour', 'activity_description': "embark on a culinary adventure through izmir's trendy alsancak district. sample local street food, visit traditional eateries, and learn about aegean cuisine.", 'min_price': '35', 'max_price': '55', 'duration': '3 hours'}),
 Document(page_content="Discover Izmir's vibrant street art scene on a guided walk through Alsancak. Learn about local artists and the stories behind their colorful murals.", metadata={'location': 'izmir', 'activity_address': 'alsancak, 35220 konak/i̇zmir', 'activity_title': 'izmir street art walk', 'activity_description': "discover izmir's vibrant street art scene on a guided walk through alsancak. learn about local artists and the storie